In [27]:
# 07_log_svm (n)
# do poniższych zadań wykorzystaj dataset z projektu.

# 1. Utwórz GridSearchCV dla pipeline, składającego się z elementów (dobierz prawidłową kolejność):
# PCA lub KPCA
# StandardScaller
# estymatory:
# LogisticRegression
# solver = ?
# penalty = ?
# class_weight = [None, "balanced"]
# C = ?
# SVC
# kernel=["linear", "rbf", "poly"]
# class_weight = [None, "balanced"]
# gamma = ?
# C = ?
# 2. Przeprowadź trening dla zdefiniowanej metryki.
# 3. Dla chętnych:
# w GridsearchCV podaj kilka metryk -> przykład
# przeprowadź analizę cv_results_

# W miejscu ? wybierz odpowiednie wartości, biorąc pod uwagę charakter danych i problem jakie ma rozwiązać algorytm ML.

# Rozwiązanie prześlij jako printscreen (jpg, pdf).
# Nazwa pliku (bez polskich znaków):

# Nazwisko_Imie_07_niestacj.jpg

In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [29]:
seed = np.random.seed(204)

In [30]:
train_X = pd.read_csv("project_data/train_data.csv", header=None)
train_label_df = pd.read_csv("project_data/train_labels.csv", header=None)

In [31]:
train_label = train_label_df[0].values

In [32]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_label, test_size=0.97, shuffle=True, random_state=seed)

In [33]:
scaler = StandardScaler().fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [57]:
# dimensionality reduction pca
pca = PCA(n_components=2, whiten=True, random_state=seed)
X_train_std_pca = pca.fit_transform(X_train_std)
X_test_std_pca = pca.fit_transform(X_test_std)

print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.00066465 0.00065751]
[157.62284193 156.77362789]


In [35]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

In [36]:
log_reg_model = LogisticRegression()
sgd_model = SGDClassifier()
svc_model = SVC()

In [37]:
pipe = Pipeline([("classifier", SVC())])

In [38]:
search_space = [
    {"classifier": [log_reg_model],
    "classifier__solver": ['lbfgs', 'sag', 'saga'],
    "classifier__penalty": ['l1', 'l2', 'elasticnet', None],
    "classifier__class_weight": ["balanced", None],
    "classifier__C": np.logspace(0, 4, 10),
    "classifier__multi_class": ['ovr']},
    
    {"classifier": [sgd_model],
    "classifier__penalty": ['l1', 'l2', 'elasticnet'],
    "classifier__class_weight": [None, "balanced"],
    "classifier__alpha": [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1], 
    "classifier__loss": ['hinge', 'log', 'perceptron']},
    
    {"classifier": [svc_model],
    "classifier__kernel":["linear", "rbf", "poly"],
    "classifier__class_weight": [None, "balanced"],
    "classifier__gamma": [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1],
    "classifier__C": np.logspace(0, 4, 10)},
]

In [39]:
skf = StratifiedKFold(n_splits=3, random_state=seed, shuffle=True)

In [40]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report

In [41]:
# scoring = {'ACC': make_scorer(accuracy_score),
#            'ROC_AUC': make_scorer(roc_auc_score)}

In [42]:
gridsearch = GridSearchCV(pipe, search_space, cv=skf, verbose=0, n_jobs=-1)

In [44]:
best_model = gridsearch.fit(X_train_std_pca, y_train)

C:\Users\DJaskulski\miniconda3\envs\cdv_data_prep\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.62612613 0.60810811 0.60810811 0.60810811
        nan        nan        nan        nan        nan        nan
        nan        nan 0.89284021 0.89284021 0.89284021 0.89284021
        nan        nan        nan        nan        nan        nan
        nan        nan 0.60810811 0.60810811 0.60810811 0.60810811
        nan        nan        nan        nan        nan        nan
        nan        nan 0.89284021 0.89284021 0.89284021 0.89284021
        nan        nan        nan        nan        nan        nan
        nan        nan 0.60810811 0.60810811 0.60810811 0.60810811
        nan        nan        nan        nan        nan        nan
        nan        nan 0.89284021 0.89284021 0.89284021 0.89284021
        nan        nan        nan        nan        nan        nan
        nan        nan 0.608

In [45]:
print(best_model.best_estimator_.get_params()["classifier"])

LogisticRegression(multi_class='ovr', penalty='l1', solver='saga')


In [46]:
gridsearch.best_params_

{'classifier': LogisticRegression(multi_class='ovr', penalty='l1', solver='saga'),
 'classifier__C': 1.0,
 'classifier__class_weight': None,
 'classifier__multi_class': 'ovr',
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [47]:
gridsearch.best_score_

0.8928402086296824

In [ ]:
# gridsearch.cv_results_

In [48]:
model = LogisticRegression(C=1, class_weight=None, multi_class='ovr', penalty='l1', solver='saga')

In [51]:
model.fit(X_train_std_pca, y_train)

LogisticRegression(C=1, multi_class='ovr', penalty='l1', solver='saga')

In [58]:
preds = model.predict(X_test_std_pca)

In [60]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       363
           1       0.90      1.00      0.95      3275

    accuracy                           0.90      3638
   macro avg       0.45      0.50      0.47      3638
weighted avg       0.81      0.90      0.85      3638



C:\Users\DJaskulski\miniconda3\envs\cdv_data_prep\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DJaskulski\miniconda3\envs\cdv_data_prep\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DJaskulski\miniconda3\envs\cdv_data_prep\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_